In [1]:
#Importing Required PAckages
import pandas as pd
import numpy as np
import re
from collections import OrderedDict
from stop_words import get_stop_words

stop = get_stop_words('english')

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Provider path
provider_path='Customer Files/'
#Positive files path
path_1= 'data_src/'
#output file path
path_2= 'data_tgt/'
#This below parametrs are core of this code 1) Provider name 2) Survey in which country
# 3) year of Survey

#provider=['quantcast','tradedesk','gdv','weborama','amazon','qualityview','tiktok']
provider='amazon'
#country=[]
country='GBR'
year=2022

In [3]:
#This code using for flatten the list in all Mapping functions
def flatten_comprehension(matrix):
    return [item for row in matrix for item in row]

In [4]:
def attributes_mapping(year,country,provider):
    #load the provider data
    if 'tradedesk'==provider:
        df=pd.read_excel(provider_path+'TradeDesk_Categories_IAB_Mapped_CAN_06 02 2023_V1.xlsx',sheet_name=1)
        df=df.iloc[:,0:8]
        cols = ['Name', 'Category / taxonomy path', 'IAB taxonomy V1 name']
    if 'quantcast'==provider:
        df=pd.read_excel(provider_path+'Quantcast_Categories_IAB_Mapped data.xlsx')
        df=df.iloc[:,0:9]
        cols = ['Category','Label']
    if 'gdv'==provider:
        df=pd.read_excel(provider_path+'googleAudience_273937.xlsx',sheet_name="googleAudience_273937")
        df=df.iloc[:,2:3]
        df = df.dropna()
        cols = ['displayName']
    if 'amazon'==provider:
        df=pd.read_excel(provider_path+'Amazon Audience Taxonomy list.xlsx')
        df=df.iloc[:,0:4]
        cols = ['Name', 'Description', 'Category']
        cols = ['Name']
    if 'weborama'==provider:
        df=pd.read_excel(provider_path+'Integration_weborama_merkury.xlsx')
        df=df.iloc[:,0:8]
        cols = ['Name (segment)', 'Weborama taxonomy V1 name (cluster)']
        
    if 'tiktok'==provider:
        df=pd.read_excel(provider_path+'TikTok.xlsx')
        df=df.iloc[:,0:2]
        cols = ['name']
        
    if 'qualityview'==provider:
        df=pd.read_excel(provider_path+'dqv_subcategories.xlsx',sheet_name='dqv_subcategories')
        df=df.iloc[:,0:1]
        cols = ['Topic_List']

        
        
    df['combined'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    #remove special chars
    df['combined']=df['combined'].str.replace('\W', ' ', regex=True)
    #Conver into lower case
    df['combined']=df['combined'].apply(lambda x:x.lower())
    #drop duplicate words
    df['combined'] = (df['combined'].str.split()
                  .apply(lambda x: OrderedDict.fromkeys(x).keys())
                  .str.join(' '))
    #load the ccs data and selected required columns
    df1=pd.read_excel(path_1+'CCS_'+str(year)+'_Positive_Codebook_'+country+'.xlsx')
    df1=df1[['Label','Variable']]
    df1.dropna(subset=['Variable'], inplace=True)
    df1.columns=['Path','AnswerId']
    df1=df1[df1['Path'].map(lambda x: x.isascii())]
    df1=df1.drop_duplicates(subset='Path')
    

    #remove special chars
    df1['Path_1']=df1['Path'].str.replace('\W', ' ', regex=True)
    #Convert into lower case
    df1['Path_1']=df1['Path_1'].apply(lambda x:str(x).lower())
    #drop duplicate words
    df1['Path_1'] = (df1['Path_1'].str.split()
                  .apply(lambda x: OrderedDict.fromkeys(x).keys())
                  .str.join(' '))
    #drop stop words
    df1['Path_1'] = df1['Path_1'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    #Gettting polarity
    df1['Str_Pol']=df1['Path'].str.split('>>').str[-1]
    df1['Str_Pol']=df1['Str_Pol'].str.lstrip()
    df1['Str_Pol']=df1['Str_Pol'].apply(lambda x:str(x).lower())
    analyzer = SentimentIntensityAnalyzer()
    df1['polarity']=df1['Str_Pol'].apply(lambda x: analyzer.polarity_scores(x)["compound"])

    #dropping no,not string answers and imported audience from all Q&A
    df1=df1[(df1["Str_Pol"].str.contains(" no | not |not ") == False)]
    df1=df1[(df1["Path_1"].str.contains("imported audience") == False)] 
    
    dropping=['never','not applicable']
    df1=df1[~df1['Str_Pol'].isin(dropping)]
    df1=df1[df1['polarity']>=0]
    df1=df1[['Path', 'AnswerId', 'Path_1']]
    
    df['key'] = 1
    df1['key'] = 1
  
    # to obtain the cross join we will merge  
    # on the key and drop it. 
    result = pd.merge(df, df1, on ='key').drop("key", axis=1) 
    training_data=df['combined'].to_list() + df1['Path_1'].to_list()
    
    a=df.shape[0]
    b=df1.shape[0]
    print('Done Data Processing')
    
    #Model Building
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    #Encoding:
    print('Model Training on the data')
    sen_embeddings = model.encode(training_data)
    print('Done Model Buiding')
    num=[]
    similarity=[]
    for i in range(a):
        num.append(i)
        similarity.append(cosine_similarity([sen_embeddings[i]],sen_embeddings[a:]))
        
    
    appended_data = []
    for i in range(a):
        c=pd.DataFrame(training_data[a:],similarity[i][0])
        appended_data.append(c)
    df2= pd.concat(appended_data)
    
    appended_data_1 = []
    for i in range(a):
        appended_data_1.append([training_data[i]]*b)
    
    flatten_Map=flatten_comprehension(appended_data_1)
    df2=df2.reset_index()
    df2.columns=['score','Path_1']
    df2['combined']=flatten_Map
    result=result.merge(df2,on=['combined','Path_1'],how='left')
    result_1=result[result['score']>=0.5]
    result_1.reset_index(drop=True,inplace=True)
    result_1.drop(['combined','Path_1'],axis=1,inplace=True)
    result_1=result_1.drop_duplicates()
    
    if 'tradedesk'==provider:
        result_1['rank']=result_1.groupby(['Segment ID', 'IAB/APP', 'Name', 'Category / taxonomy path',
                           'IAB taxonomy V1 name', 'IAB taxonomy V1 ID', 'Aggr A',
                          'Variables for modelling'])['score'].rank(method='first',ascending=False)
        result_2=result_1[result_1['rank']<=30]
    
        result_3=result_2.pivot(index=['Segment ID', 'IAB/APP', 'Name', 'Category / taxonomy path',
                           'IAB taxonomy V1 name', 'IAB taxonomy V1 ID', 'Aggr A','Variables for modelling'],
                  columns='rank',values=['Path','AnswerId'])
        
    if 'quantcast'==provider:
        result_1['rank']=result_1.groupby(['order', 'IAB/APP', 'Category', 'Aggr A'
                                  ])['score'].rank(method='first',ascending=False)
        result_2=result_1[result_1['rank']<=30]
    
        result_3=result_2.pivot(index=['order', 'IAB/APP', 'Category', 'Label', 'Aggr A', 'IABs to aggregate',
                         'Unnamed: 6', 'Aggr C', 'Vars for modelling'],
               columns='rank',values=['Path','AnswerId'])
        
    if 'gdv'==provider:
        result_1['rank']=result_1.groupby(['displayName'
                                  ])['score'].rank(method='first',ascending=False)
        result_2=result_1[result_1['rank']<=30]
    
        result_3=result_2.pivot(index=['displayName'],
               columns='rank',values=['Path','AnswerId'])
    
    if 'amazon'==provider:
        result_1['rank']=result_1.groupby(['Name', 'Description', 'Category'
                                  ])['score'].rank(method='first',ascending=False)
        result_2=result_1[result_1['rank']<=30]
        result_3=result_2.pivot(index=['Name', 'Description', 'Category','Audience Id'],
                    columns='rank',values=['Path','AnswerId'])
        
    if 'weborama'==provider:
        result_1['rank']=result_1.groupby(['Name (segment)', 'Category / taxonomy path (kws)',
                                           'Weborama taxonomy V1 name (cluster)'
                                           ])['score'].rank(method='first',ascending=False)
        result_2=result_1[result_1['rank']<=30]
        result_3=result_2.pivot(index=['Segment ID', 'IAB/APP', 'Name (segment)','Category / taxonomy path (kws)',
                                       'Weborama taxonomy V1 name (cluster)','IAB taxonomy V1 ID (id del segmentoPEDIR)', 
                                       'Aggr A','Variables for modelling'],columns='rank',values=['Path','AnswerId'])
    if 'tiktok'==provider:
        result_1['rank']=result_1.groupby(['id','name'
                                           ])['score'].rank(method='first',ascending=False)
        result_2=result_1[result_1['rank']<=30]
        result_3=result_2.pivot(index=['id','name'],columns='rank',values=['Path','AnswerId'])
        
    if 'qualityview'==provider:
        result_1['rank']=result_1.groupby(['Topic_List'
                                  ])['score'].rank(method='first',ascending=False)
        result_2=result_1[result_1['rank']<=30]
    
        result_3=result_2.pivot(index=['Topic_List'],
               columns='rank',values=['Path','AnswerId'])
           
        
    #format column names
    result_3.columns = ['_'.join(str(s).strip() for s in col if s) for col in result_3.columns]

    #reset index
    result_3.reset_index(inplace=True)
    
    if 'tradedesk'==provider:
        result_3.rename(columns={'Name':'Label','Category / taxonomy path':'Category', 
        'AnswerId_1.0':'Aggr B - CCS vars to aggregate (positions 1 to 223)'},inplace=True)
        
    if 'quantcast'==provider:
        result_3.rename(columns={'Unnamed: 6':'','AnswerId_1.0':'Aggr B - CCS vars to aggregate (positions 1 to 223)'},
                    inplace=True)
    if 'gdv'==provider:
        result_3.rename(columns={'AnswerId_1.0':'Aggr B - CCS vars to aggregate (positions 1 to 223)',
                                 'displayName':'Label'},inplace=True)
    if 'amazon'==provider:
        result_3.rename(columns={'AnswerId_1.0':'Aggr B - CCS vars to aggregate (positions 1 to 223)'},
                    inplace=True)
    if 'weborama'==provider:
        result_3.rename(columns={'Name (segment)':'Label','Weborama taxonomy V1 name (cluster)':'Category', 
        'AnswerId_1.0':'Aggr B - CCS vars to aggregate (positions 1 to 223)'},inplace=True)
        
    if 'tiktok'==provider:
        result_3.rename(columns={'AnswerId_1.0':'Aggr B - CCS vars to aggregate (positions 1 to 223)'},
                        inplace=True)
    if 'qualityview'==provider:
        result_3.rename(columns={'AnswerId_1.0':'Aggr B - CCS vars to aggregate (positions 1 to 223)'},
                        inplace=True)
        
        
        
    result_3.to_excel(path_2+provider+'_Attributes_'+country+'_'+str(year)+'_QA_cosine.xlsx',index=False)
    
    
    return result_3

In [5]:
result=attributes_mapping(year,country,provider)

Done Data Processing
Model Training on the data
Done Model Buiding


In [6]:
result

,Name,Description,Category,Audience Id,Path_1.0,Path_2.0,Path_3.0,Path_4.0,Path_5.0,Path_6.0,Path_7.0,Path_8.0,Path_9.0,Path_10.0,Path_11.0,Path_12.0,Path_13.0,Path_14.0,Path_15.0,Path_16.0,Path_17.0,Path_18.0,Path_19.0,Path_20.0,Path_21.0,Path_22.0,Path_23.0,Path_24.0,Path_25.0,Path_26.0,Path_27.0,Path_28.0,Path_29.0,Path_30.0,Aggr B - CCS vars to aggregate (positions 1 to 223),AnswerId_2.0,AnswerId_3.0,AnswerId_4.0,AnswerId_5.0,AnswerId_6.0,AnswerId_7.0,AnswerId_8.0,AnswerId_9.0,AnswerId_10.0,AnswerId_11.0,AnswerId_12.0,AnswerId_13.0,AnswerId_14.0,AnswerId_15.0,AnswerId_16.0,AnswerId_17.0,AnswerId_18.0,AnswerId_19.0,AnswerId_20.0,AnswerId_21.0,AnswerId_22.0,AnswerId_23.0,AnswerId_24.0,AnswerId_25.0,AnswerId_26.0,AnswerId_27.0,AnswerId_28.0,AnswerId_29.0,AnswerId_30.0
0,4K DVD Fans,People who are 4k DVD fans,Experimental,s=30374,Film 4 - GBR - Which of these TV channels are ...,More 4 - GBR - Which of these TV channels are ...,Film 4 - GBR - Which of these TV channels do y...,4seven - GBR - Which of these TV channels are ...,More 4 - GBR - Which of these TV channels do y...,Channel 4 - GBR - Which of these TV channels a...,4 Music - GBR - Which of these TV channels do ...,4seven - GBR - Which of these TV channels do y...,4 Music - GBR - Which of these TV channels are...,Channel 4 - GBR - Which of these TV channels d...,Channel 4 - IRE - Which of these TV channels d...,Film4 - IRE - Which of these TV channels do yo...,E4 - GBR - Which of these TV channels do you R...,Went to the cinema recently - BASE for Z4 : Yes,Channel 4 - IRE - Which of these TV channels a...,E4 - GBR - Which of these TV channels are your...,E4 - IRE - Which of these TV channels do you R...,More4 - IRE - Which of these TV channels do yo...,TV5 - CAN - Which of these TV channels are you...,Film4 - IRE - Which of these TV channels are y...,TV5 - CAN - Which of these TV channels do you ...,All 4 - GBR - How many TV PROGRAMMES or FILMS ...,More4 - IRE - Which of these TV channels are y...,5 Star - GBR - Which of these TV channels do y...,TG4 - IRE - Which of these TV channels do you ...,5ACTION - GBR - Which of these TV channels do ...,Channel 4 News - GBR - Which of these TV progr...,Channel 5 - GBR - Which of these TV channels d...,5SELECT - GBR - Which of these TV channels do ...,E4 - IRE - Which of these TV channels are your...,W2b_25,W2b_41,W2a_25,W2b_2,W2a_41,W2b_16,W2a_1,W2a_2,W2b_1,W2a_16,W2a_612,W2a_619,W2a_24,BASE_Z4_13,W2b_612,W2b_24,W2a_618,W2a_622,W2b_734,W2b_619,W2a_734,W6_9,W2b_622,W2a_3,W2a_641,W2a_5,W3a_17,W2a_17,W2a_6,W2b_618
1,Demo - Affluence: Average,Of average affluence,Demographic,s=49852,I am very happy with my standard of living - M...,MAINSTREAM_LUXURY : OTHER,I like things less when they become mainstream...,MAINSTREAM_LUXURY : MAINSTREAM,To Be Alone - Sub Motivations : Yes,I like things less when they become mainstream...,I like things less when they become mainstream...,I am very happy with my current relationship s...,I hold unconventional views - Self - ANY AGREE...,Holiday - I am confident I make the right choi...,I have a very active imagination - Self - ANY ...,MAINSTREAM_LUXURY : LUXURY,DAB+ - At least once a day (NET) - How often d...,How old are you? : 45,DMA : YUMA-EL CENTRO,I like things less when they become mainstream...,Tiny Pop - GBR - Which of these TV channels ar...,How old are you? : 51,I want to be popular - Self - ANY DISAGREE : Yes,I hold unconventional views - Self : Somewhat ...,Alone - Who do you live with? : Yes,Holiday - It is very rewarding to make a good ...,High street - At least once a month (NET) : Yes,Holiday - I give this very little thought - Ho...,Qualification Segments : Intermediate qualific...,"If there is something I want, I save up for it...",I like to do things my own way and make my own...,How old are you? : 46,How old are you? : 53,I tend to get stressed out very easily - Self ...,A6_AD_13,G3_R4,A11_AA_7,G3_R4,MOT_R2_25,A11_7,A11_AD_7,A4_AD_12,A7_AA_17,Z12_16_AD_79,A3_AD_20,G3_R4,L1_R1a_13,

In [7]:
result.shape

(843, 64)